In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import requests
import bs4 as bs
import pickle
from matplotlib import pyplot as plt

In [2]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)

    with open("../sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)

    return tickers
tickers = save_sp500_tickers()
for i in range (len(tickers)):
    tickers[i]=tickers[i].replace('\n', '')
print(len(tickers))
print(tickers)

503
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI

In [11]:
def define_stocks(start_date, end_date ,number_of_stocks):
    raw_data = yf.download(tickers[0:number_of_stocks], start=start_date, end=end_date, interval="1d")
    raw_data_close = raw_data['Close']

    # remove stocks that have one or more missing values
    data = raw_data_close.dropna(axis=1)
    remaining_stocks = len(data.columns)
    stocks_removed = number_of_stocks - remaining_stocks

    # Print a message to say how many stocks were removed
    print("Stocks removed are:", stocks_removed)
    print("Remaining stocks are:", remaining_stocks)

    return data

In [23]:
stock_data = define_stocks("2016-01-01", "2024-06-25", 503)
# stock_data.head()

[*********************100%%**********************]  503 of 503 completed

8 Failed downloads:
['RJF', 'MSCI', 'CMS', 'ORCL', 'HON', 'BSX']: ConnectionError(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out."))
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2016-01-01 -> 2024-06-24)')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Stocks removed are: 32
Remaining stocks are: 471


In [13]:
print(stock_data.shape)
stock_data.to_csv("../data/raw_data.csv")
stock_data.describe().T

(2109, 478)


,count,mean,std,min,25%,50%,75%,max
Ticker,,,,,,,,
A,2109.0,95.000631,37.426942,34.799999,65.750000,82.930000,129.880005,179.279999
AAL,2109.0,27.468160,13.316499,9.040000,14.870000,24.230000,38.759998,58.470001
AAPL,2109.0,94.865173,57.665658,22.584999,42.259998,74.389999,149.399994,198.110001
ABBV,2109.0,104.583983,34.314967,51.180000,76.949997,96.059998,138.279999,182.100006
ABT,2109.0,84.753400,28.813370,36.340000,58.820000,88.230003,109.269997,141.460007
...,...,...,...,...,...,...,...,...
XYL,2109.0,82.685258,25.084863,32.799999,66.029999,79.610001,102.290001,145.770004
YUM,2109.0,99.577078,25.074788,46.901508,79.430000,100.900002,121.839996,143.190002
ZBH,2109.0,122.998550,15.633024,78.281555,112.563110,120.650002,130.611649,173.407761


In [14]:
data_place_holder = stock_data
n_row = len(data_place_holder)
## Empty data frame to fill with the return series
data_return = pd.DataFrame(index=range(n_row),columns=range(len(data_place_holder.columns)))
## Loop through each stock and calculate its return series
for i in range(len(data_place_holder.columns)):
    for j in range(n_row-1):
        data_return.iloc[j+1,i] = (data_place_holder.iloc[j+1,i] - data_place_holder.iloc[j,i]) / data_place_holder.iloc[j,i]

data_return.head()

,0,1,2,3,4,5,6,7,8,9,...,468,469,470,471,472,473,474,475,476,477
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.003441,-0.009533,-0.025059,-0.004166,-0.000233,0.003631,0.005205,0.004023,-0.007347,0.011161,...,0.00688,0.002347,0.018904,0.010084,0.008521,-0.000277,-0.002493,0.020823,-0.025117,0.015655
2,0.004439,0.017522,-0.01957,0.000174,-0.008388,0.001592,-0.001954,-0.014295,-0.042561,-0.024283,...,0.0,-0.022074,-0.051805,0.010538,-0.008321,-0.012476,-0.00708,0.00433,-0.039957,0.000208
3,-0.042475,-0.018918,-0.042205,-0.002963,-0.023966,-0.001011,-0.029366,-0.020984,-0.025512,-0.023756,...,-0.086538,-0.036936,-0.09407,0.003842,-0.016006,-0.025828,-0.034116,-0.022993,-0.045316,-0.030404
4,-0.010513,-0.001978,0.005288,-0.027268,-0.020944,-0.010414,-0.009681,-0.01414,-0.008727,-0.003766,...,0.014528,-0.009233,-0.040704,-0.010935,-0.020202,-0.00951,-0.013607,-0.004217,-0.002693,-0.014605


In [15]:
normalized_return = (data_return - data_return.mean()) / data_return.std()
# normalized_return.to_csv("./data/normalized_return.csv")
normalized_return.head()

,0,1,2,3,4,5,6,7,8,9,...,468,469,470,471,472,473,474,475,476,477
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.248163,-0.288812,-1.428912,-0.291923,-0.050395,0.157723,0.285137,0.141127,-0.419008,0.62438,...,0.412599,0.100152,0.583809,0.707926,0.447067,-0.062973,-0.211262,1.163966,-1.042604,0.913124
2,0.21532,0.526781,-1.129142,-0.027978,-0.587979,0.041869,-0.162921,-0.718857,-2.219129,-1.432763,...,-0.029298,-1.060559,-1.678196,0.740799,-0.476525,-0.767853,-0.527344,0.231077,-1.633636,-0.033369
3,-2.544178,-0.571731,-2.365155,-0.218752,-1.614974,-0.106037,-1.878631,-1.032905,-1.347592,-1.402179,...,-5.587596,-1.766968,-3.030255,0.25582,-0.898016,-1.539407,-2.390076,-1.314335,-1.847063,-1.90917
4,-0.664153,-0.061054,0.228236,-1.696812,-1.415724,-0.640229,-0.646588,-0.711573,-0.489511,-0.241971,...,0.903831,-0.450235,-1.323073,-0.814468,-1.128123,-0.596488,-0.977047,-0.252319,-0.149566,-0.94106


In [16]:
transposed_data = normalized_return.transpose()
transposed_data.index = [list(stock_data.columns)]
del transposed_data[0]
transposed_data.to_csv("../data/processed_data.csv")
transposed_data.head()

,1,2,3,4,5,6,7,8,9,10,...,2099,2100,2101,2102,2103,2104,2105,2106,2107,2108
A,-0.248163,0.21532,-2.544178,-0.664153,-1.036544,0.341807,-2.094259,1.151056,-0.827764,0.096871,...,0.230313,0.591263,0.918083,1.826554,-0.831314,1.355468,1.042913,0.042019,-0.076278,0.110586
AAL,-0.288812,0.526781,-0.571731,-0.061054,0.528747,0.673683,-1.365156,0.336858,-1.332144,0.076344,...,-0.824516,0.358197,0.123998,-0.167982,1.108088,0.523537,-0.497572,-0.304097,-0.123727,-0.431203
AAPL,-1.428912,-1.129142,-2.365155,0.228236,0.823694,0.732015,-1.464459,1.133781,-1.3719,-0.32474,...,0.146843,0.041283,0.486334,-0.510215,0.903048,0.276603,0.606671,-0.025971,-0.05188,0.275982
ABBV,-0.291923,-0.027978,-0.218752,-1.696812,-1.972781,1.044939,-4.097187,3.97757,3.060007,-2.530897,...,-0.105844,-0.824226,-0.057532,0.094118,0.161921,0.078308,0.789364,0.169344,0.727356,-0.718254
ABT,-0.050395,-0.587979,-1.614974,-1.415724,0.062223,1.146507,-1.911301,1.307005,-0.933262,-0.213912,...,0.295705,-0.798772,-0.204654,0.009052,-0.016155,-0.513242,0.332576,0.141454,-0.525366,-0.592368
